# Create trialParams.nc
Creates empty `trialParams.nc` file for SUMMA runs. This file will initially be empty, but can later be populated with parameter values the user whishes to test.

## Note on HRU order
HRU order must be the same in forcing, attributes, initial conditions and trial parameter files. Order will be taken from forcing files to ensure consistency.

In [2]:
# modules
import os
os.chdir('/home/dlhogan/GitHub/mtn_hydro_pysumma')
import numpy as np
import xarray as xr
import netCDF4 as nc4
from pathlib import Path
from shutil import copyfile
from datetime import datetime
from utils.read_files import read_from_control, make_default_path

#### Control file handling

In [3]:
# Store the name of the 'active' file in a variable
basin = "EastRiver"
controlFile = f'control_{basin}.txt'

#### Find forcing location and an example file

In [4]:
# Forcing path
forcing_path = read_from_control(controlFile,'forcing_summa_path')

In [5]:
# Specify default path if needed
if forcing_path == 'default':
    forcing_path = make_default_path('forcing/4_SUMMA_input', controlFile) # outputs a Path()
else:
    forcing_path = Path(forcing_path) # make sure a user-specified path is a Path()

In [6]:
# Find a list of forcing files
_,_,forcing_files = next(os.walk(forcing_path))

In [7]:
# Select a random file as a template for hruId order
forcing_name = forcing_files[0]

#### Find where the trial parameter file needs to go

In [8]:
# Trial parameter path & name
parameter_path = read_from_control(controlFile,'settings_summa_path')
parameter_name = read_from_control(controlFile,'settings_summa_trialParams')

In [9]:
# Specify default path if needed
if parameter_path == 'default':
    parameter_path = make_default_path('settings/SUMMA', controlFile) # outputs a Path()
else:
    parameter_path = Path(parameter_path) # make sure a user-specified path is a Path()

In [10]:
# Make the folder if it doesn't exist
parameter_path.mkdir(parents=True, exist_ok=True)

#### Find order and number of HRUs in forcing file

In [11]:
# Open the forcing file
forc = xr.open_dataset(forcing_path/forcing_name)

/tmp/ipykernel_25612/1115369196.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  forc = xr.open_dataset(forcing_path/forcing_name)


In [12]:
# Get the sorting order from the forcing file
forcing_hruIds = forc['hruId'].values.astype(int) # 'hruId' is prescribed by SUMMA so this variable must exist

In [13]:
# Number of HRUs
num_hru = len(forcing_hruIds)

#### Read any other trial parameters that need to be specified

In [14]:
num_tp = int( read_from_control(controlFile,'settings_summa_trialParam_n') )

In [15]:
# read the names and values of trial parameters to specify
all_tp = {}
for ii in range(0,num_tp):
    
    # Get the values
    par_and_val = read_from_control(controlFile,f'settings_summa_trialParam_{ii+1}')
    
    # Split into parameter and value
    arr = par_and_val.split(',')
    
    # Convert value(s) into float
    if len(arr) > 2:
        # Store all values as an array of floats    
        val = np.array(arr[1:], dtype=np.float32)
    else: 
        # Convert the single value to a float
        val = float( arr[1] )
        
    # Store in dictionary
    all_tp[arr[0]] = val

#### Make the trial parameter file

In [16]:
# Create the empty trial params file
with nc4.Dataset(parameter_path/parameter_name, "w", format="NETCDF4") as tp:
    
    # === Some general attributes
    now = datetime.now()
    tp.setncattr('Author', "Created by SUMMA workflow scripts")
    tp.setncattr('History','Created ' + now.strftime('%Y/%m/%d %H:%M:%S'))
    tp.setncattr('Purpose','Create a trial parameter .nc file for initial SUMMA runs')
    
    # === Define the dimensions 
    tp.createDimension('hru',num_hru)
    
    # === Variables ===
    var = 'hruId'
    tp.createVariable(var, 'i4', 'hru', fill_value = False)
    tp[var].setncattr('units', '-')
    tp[var].setncattr('long_name', 'Index of hydrological response unit (HRU)')
    tp[var][:] = forcing_hruIds
    
    # Loop over any specified trial parameters and store in file
    for var,val in all_tp.items():
        tp.createVariable(var, 'f8', 'hru', fill_value = False)
        tp[var][:] = val    

In [17]:
parameter_path

PosixPath('/storage/dlhogan/summa_modeling_data/domain_EastRiver/settings/SUMMA')